In [1]:
# Imports here
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models
import time
import numpy as np
from PIL import Image
import json
from collections import OrderedDict

# data_dir = 'flowers'
data_dir = '~/.pytorch/flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

# TODO: Define your transforms for the training, validation, and testing sets
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                         transforms.RandomResizedCrop(224),
                                         transforms.RandomHorizontalFlip(p = 0.6),
                                         transforms.RandomVerticalFlip(p = 0.4),
                                         transforms.ToTensor(),
                                         transforms.Normalize([0.485, 0.456, 0.406],
                                                              [0.229, 0.224, 0.225])])
test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])
valid_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# TODO: Load the datasets with ImageFolder
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
test_data = datasets.ImageFolder(test_dir, transform=test_transforms)
valid_data = datasets.ImageFolder(valid_dir, transform=valid_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=64)

import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
    
num_output_classes = len(cat_to_name)
num_output_classes

102

In [2]:
model = models.resnet50(pretrained=True)
model


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [78]:
classifier_layer = list(model.children())[-1]
classifier_layer.in_features

2048

In [104]:
classifier_layer.named_parameters()

        
        
l = []
x = [l.append(name) for name, param in model.named_parameters()]
last_layer = l[-1]
print("classifier" in last_layer)
print("fc" in last_layer)

print(last_layer.split('.')[0])
print("classifier" == last_layer)
print("fc" == last_layer)

True
False
classifier
False
False


In [79]:
model = models.densenet121(pretrained=True)
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [81]:


classifier_layer = list(model.children())[-1]
classifier_layer.in_features
classifier_layer

Linear(in_features=1024, out_features=1000, bias=True)

In [83]:
for name, param in model.named_parameters():
        print(name)

AttributeError: 'DenseNet' object has no attribute 'attributes'

In [45]:

input_size = 784
output_size = 102
hidden_layers = [512, 256, 128]
layers = hidden_layers[:]
layers.insert(0, input_size)
layers


[784, 512, 256, 128]

In [41]:
layers_sizes = zip(layers[:-1],layers[1:])

for h1, h2 in layers_sizes:
    print(h1, h2)



784 512
512 256
256 128
128 102


In [66]:
class ClassifierNetwork(nn.Module):
    def __init__(self, input_size, output_size, hidden_layers, drop_prob=0.2):
        super().__init__()

        layers_sizes = 784, 10, [512, 256, 128]

        input_size = 784
        output_size = 102
        hidden_layers = [512, 256, 128]

        # create input layer        
        self.hidden_layers = nn.ModuleList([nn.Linear(input_size,hidden_layers[0])])

        # add hidden layer to input layer
        layers = hidden_layers[:]
        layers_sizes = zip(layers[:-1], layers[1:])
        self.hidden_layers.extend([nn.Linear(i, o) for i, o in layers_sizes])
        
        #Create output layer
        self.output_layer = nn.Linear(hidden_layers[-1], output_size)


In [67]:

m1 = ClassifierNetwork(784, [512, 256, 128], 102)
m1

ClassifierNetwork(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=256, bias=True)
    (2): Linear(in_features=256, out_features=128, bias=True)
  )
  (output_layer): Linear(in_features=128, out_features=102, bias=True)
)